In [ ]:
import pandas as pd
import numpy as np

prot_df = pd.read_csv("/Users/vitran/Documents/Work/Github/Insectipep/data/final_target_list.csv")
for _,row in prot_df.iterrows():
    seq = row["Sequence"]
    if len(seq) < 50:
        prot_df.at[_, "Group"] = "S"
    elif len(seq) < 150:
        prot_df.at[_, "Group"] = "M"
    else:
        prot_df.at[_, "Group"] = "L"


In [ ]:
import os
import shutil
import pandas as pd

# Load the dataframe
prot_df = pd.read_csv("/Users/vitran/Documents/Work/Github/Insectipep/data/final_target_list.csv")

# Classify by sequence length
def classify_len(seq):
    l = len(seq)
    if l <= 400:
        return "S"
    elif l <= 700:
        return "M"
    else:
        return "L"

prot_df["Group"] = prot_df["Sequence"].apply(classify_len)

# Set chunk sizes
chunk_sizes = {"S": 60, "M": 30, "L": 15}

# Folder containing .fa files
fa_folder = "/Users/vitran/Documents/Work/Github/ColabFold_data/insectipep"
output_base = "/Users/vitran/Documents/Work/Github/ColabFold_data/insectipep/split"

# Ensure output base exists
os.makedirs(output_base, exist_ok=True)

for group in ["S", "M", "L"]:
    # Get pdbs for this group
    pdbs = prot_df[prot_df["Group"] == group]["PDB"].tolist()
    # Find matching .fa files
    fa_files = [f for f in os.listdir(fa_folder) if f.endswith(".fa") and any(pdb in f for pdb in pdbs)]
    # Split into chunks
    chunk_size = chunk_sizes[group]
    for i in range(0, len(fa_files), chunk_size):
        folder_name = f"{group}_{i//chunk_size+1}"
        folder_path = os.path.join(output_base, folder_name)
        os.makedirs(folder_path, exist_ok=True)
        for fa_file in fa_files[i:i+chunk_size]:
            shutil.copy(os.path.join(fa_folder, fa_file), folder_path)

In [14]:
import os

parent_folder = './insectipep'

for subfolder in os.listdir(parent_folder):
    
    subfolder_path = os.path.join(parent_folder, subfolder)
    
    if os.path.isdir(subfolder_path):
        num_files = len([
            f for f in os.listdir(subfolder_path)
            if os.path.isfile(os.path.join(subfolder_path, f))
        ])
        print(f"{subfolder}: {num_files} files")
        number += num_files
print(f"Total files: {number}")

split: 0 files
Total files: 5768


In [11]:
import os
import shutil
import pandas as pd

# Load the dataframe
prot_df = pd.read_csv("/Users/vitran/Documents/Work/Github/Insectipep/data/final_target_list.csv")

# Classify by sequence length
def classify_len(seq):
    l = len(seq)
    if l <= 400:
        return "S"
    elif l <= 700:
        return "M"
    else:
        return "L"

prot_df["Group"] = prot_df["Sequence"].apply(classify_len)

# Set chunk sizes
chunk_sizes = {"S": 60, "M": 30, "L": 15}

# Folder containing .fa files
fa_folder = "/Users/vitran/Documents/Work/Github/ColabFold_data/insectipep"
output_base = "/Users/vitran/Documents/Work/Github/ColabFold_data/insectipep/split"

# Ensure output base exists
os.makedirs(output_base, exist_ok=True)

for group in ["S", "M", "L"]:
    # Get pdbs for this group
    pdbs = prot_df[prot_df["Group"] == group]["PDB"].tolist()
    # Find matching .fa files
    fa_files = [f for f in os.listdir(fa_folder) if f.endswith(".fa") and any(pdb in f for pdb in pdbs)]
    # Split into chunks
    chunk_size = chunk_sizes[group]
    for i in range(0, len(fa_files), chunk_size):
        folder_name = f"{group}_{i//chunk_size+1}"
        folder_path = os.path.join(output_base, folder_name)
        os.makedirs(folder_path, exist_ok=True)
        for fa_file in fa_files[i:i+chunk_size]:
            src = os.path.join(fa_folder, fa_file)
            dst = folder_path
            try:
                shutil.move(src, dst)
            except FileNotFoundError:
                pass